In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=(6,6)
plt.rcParams['font.weight']='bold'
plt.rcParams['axes.labelweight']='bold'
plt.rcParams['lines.linewidth']=2
plt.rcParams['lines.markeredgewidth']=2

sys.path.append('/home/roy/AtomCraft/install_release/python')
from OpenFUSIONToolkit.TokaMaker import TokaMaker
from OpenFUSIONToolkit.TokaMaker.meshing import load_gs_mesh
from OpenFUSIONToolkit.TokaMaker.util import create_power_flux_fun

mygs = TokaMaker()

#----------------------------------------------
Open FUSION Toolkit Initialized
Development branch:   main
Revision id:          aba6d8f
Parallelization Info:
  Not compiled with MPI
  # of OpenMP threads =    2
Fortran input file    = oftpyin                                                                         
XML input file        = none                                                                            
Integer Precisions    =    4   8
Float Precisions      =    4   8  10
Complex Precisions    =    4   8
LA backend            = native
#----------------------------------------------



## Inverse Equilibrium

In [2]:
mesh_name = 'ATOMCRAFT_mesh1'
B0 = 0.1 # T
R0 = 0.3 # m
Z0 = 0 # m
a = 0.1 # m
kappa = 1.3 # elongation
delta = 0 # triangularity
xi = 0.0 # squareness
Ip_target = 10e3 # A
P0_target = 1e2 # Pa

# Define last close flux surface
theta = np.linspace(0, 2 * np.pi, 10)
R = R0 + a * np.cos(theta + delta * np.sin(theta) - xi * np.sin(2 * theta))
Z = Z0 + kappa * a * np.sin(theta + xi * np.sin(2 * theta))
LCFS = np.stack([R, Z], axis=1)

In [ ]:
mygs.reset()

mesh_pts ,mesh_lc, mesh_reg, coil_dict, cond_dict = load_gs_mesh(f'meshes/{mesh_name}.h5')
mygs.setup_mesh(mesh_pts, mesh_lc, mesh_reg)
mygs.setup_regions(cond_dict=cond_dict, coil_dict=coil_dict)
mygs.setup(order = 2, F0 = B0 * R0)

coil_bounds = np.zeros((mygs.ncoils+1,2), dtype=np.float64)
coil_bounds[:,0] = -1e7; coil_bounds[:,1] = 1e7

ffp_prof = create_power_flux_fun(40,1.5,2.0)
pp_prof = create_power_flux_fun(40,4.0,1.0)

mygs.set_coil_bounds(coil_bounds)
mygs.set_targets(Ip=Ip_target, pax=P0_target)
mygs.set_isoflux(LCFS)
mygs.set_profiles(ffp_prof=ffp_prof, pp_prof=pp_prof)

mygs.settings.maxits = 100
mygs.update_settings()
err_flag = mygs.init_psi(R0, Z0, a, kappa, delta)
err_flag = mygs.solve()

In [ ]:
fig, ax = plt.subplots(1,1)
mygs.plot_machine(fig,ax,coil_colormap='seismic',coil_symmap=True,coil_scale=1.E-3,coil_clabel=r'$I_C$ [kA]')
mygs.plot_psi(fig,ax,xpoint_color=None,vacuum_nlevels=4)
mygs.plot_constraints(fig,ax,isoflux_color='tab:red',isoflux_marker='o')

In [ ]:
mygs.print_info()

print()
print("Coil Currents [kA]:")
coil_currents, _ = mygs.get_coil_currents()
for key in mygs.coil_sets:
    i = mygs.coil_sets[key]['id']
    print('  {0:10} {1:10.2F}'.format(key+":",coil_currents[i]/1.E3))

## Forward Equilibrium

In [ ]:
mygs.set_isoflux(None)
mygs.set_coil_currents(coil_currents)

mygs.set_targets(Ip=8e3) # change plasma current

err_flag = mygs.solve()

In [ ]:
fig, ax = plt.subplots(1,1)
mygs.plot_machine(fig,ax,coil_colormap='seismic',coil_symmap=True,coil_scale=1.E-3,coil_clabel=r'$I_C$ [kA]')
mygs.plot_psi(fig,ax,xpoint_color=None,vacuum_nlevels=4)

mygs.print_info()